In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd

path = '/winequalityred.csv'

df = pd.read_csv(path)               #veri setini okunması ve pandas dataframe e dönüşümü

print(df.head())                      #veri setinin ilk 5 satırını ekrana yazdırıldı
print(df.info())                      #veri seti hakkında temel bilgileri ekrana yazdırıldı

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(9, 9))                #özelliklerin dağılımını görselleştirme  figür boyutu 9,9 olarak seçildi
sns.pairplot(df, diag_kind='kde')         #veri setindeki değişkenler arasındaki ilişkiyi görselleştirir (kde kernel density estimation görselleştirme için kullanıldı)
plt.show()

plt.figure(figsize=(9, 9))                    #figür oluşturulması boyut 9,9 olarak belirlendi
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')    #veri setindeki değişkenler arasındaki korelasyonu ısı haritasının oluşturuldu
plt.title('Korelasyon Matrisi')
plt.show()

x = df.drop('quality', axis=1)         #bağımsız değişkenler için DataFrame
y = df['quality']                      #hedef değişken quality sütunu


df_encoded = pd.get_dummies(x)     #one hot encoding işlemi uyguladı

x = pd.concat([x.drop(columns=df_encoded.columns), df_encoded], axis=1)    #orijinal kategorik sütunlar kodlanmışlarla değiştirildi

from sklearn.preprocessing import StandardScaler            #normalleştirme için standartscaler
from sklearn.model_selection import train_test_split        #eğitim ve test setlerine bölme için train_test_split


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)           #normalleştirme işlemi


x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)  #eğitim ve test setlerine bölme
#veri setinin %20 si test seti %80 eğitim seti olmuş oldu
# random state rastgele bölünmeyi kontrol eder genelde 42 değeri
# kullanılıyor

from sklearn.ensemble import RandomForestClassifier         #Random Forest,Logistic Regression, K-Nearest Neighbors, Support Vector Machine
#sınıflandırma modellerine çapraz doğrulama kullanarak performanslarını inceledim, ortalama doğruluk değeri en yüksk olan random forest oldu
#bu modelden devam edildi
from sklearn.model_selection import GridSearchCV       #hiperparametre optimizasyonu için gridsearchcv

param_grid = {
    'n_estimators': [50, 100, 150],  #randomforest modelinde ağaç sayısının belirler
    'max_depth': [None, 5, 10],      #her bir ağacın maksimum derinliğini belirler
    'min_samples_split': [2, 5, 10]  #bölme için min örnekleme sayısı belirler
}


grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)   #5 katlı çapraz doğrulama yapıldı
grid_search.fit(x_train, y_train)                                        #grid search ile en iyi model bulundu

print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

best_model = grid_search.best_estimator_     #en iyi model seçildi

y_pred = best_model.predict(x_test)       #test seti tahmini


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)                            #beklenen değerlerin hesabı
class_report = classification_report(y_test, y_pred)                      #sınıflandırma raporu

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)                        #confusion matrix oluşturuldu değerler ekrana yazdırıldı
print("Classification Report:")
print(class_report)

Output hidden; open in https://colab.research.google.com to view.